In [4]:
#행정동 법정동 매핑 데이터프레임 불러오기

import pandas as pd


mapping_bh_dong = pd.read_csv('clean_map_dh_dong.csv', index_col = 0)
mapping_bh_dong

,시군구명,행정동코드,행정동명,법정동코드,법정동명,법정동개수
0,종로구,11110515,청운효자동,11110101,청운동,10
1,종로구,11110515,청운효자동,11110102,신교동,10
2,종로구,11110515,청운효자동,11110103,궁정동,10
3,종로구,11110515,청운효자동,11110104,효자동,10
4,종로구,11110515,청운효자동,11110105,창성동,10
...,...,...,...,...,...,...
736,강동구,11740650,성내제2동,11740108,성내동,1
737,강동구,11740660,성내제3동,11740108,성내동,1
738,강동구,11740685,길동,11740105,길동,1
739,강동구,11740690,둔촌제1동,11740106,둔촌동,1


In [9]:

yearmonth = ['201911','202005','202009','202101','202106','202110','202201','202204']
yearmonth = ['201911','202005']
# yearmonth = ['202009','202101','202106','202110','202201','202204']
for i in yearmonth:

    move_df = pd.read_csv('move'+i+'.csv', index_col = False)

    # 잘못된 행정코드 고치기
    move_df['행정동코드'].replace([11305610, 11305600, 11305630, 11305606, 11305590, 11305620], [11305615, 11305603, 11305635, 11305608,11305595,11305625], inplace = True)

    #연령대 10대 기준으로 변경
    move_df_clean = move_df.iloc[:,:4]
    move_df_clean['남성 10대 미만'] = move_df.iloc[:,4]
    move_df_clean['남성 10대 '] = move_df.iloc[:,5]+move_df.iloc[:,6]
    move_df_clean['남성 20대 '] = move_df.iloc[:,7]+move_df.iloc[:,8]
    move_df_clean['남성 30대 '] = move_df.iloc[:,9]+move_df.iloc[:,10]
    move_df_clean['남성 40대 '] = move_df.iloc[:,11]+move_df.iloc[:,12]
    move_df_clean['남성 50대 '] = move_df.iloc[:,13]+move_df.iloc[:,14]
    move_df_clean['남성 60대 '] = move_df.iloc[:,15]+move_df.iloc[:,16]
    move_df_clean['남성 70대 이상'] = move_df.iloc[:,17]

    move_df_clean['여성 10대 미만'] = move_df.iloc[:,18]
    move_df_clean['여성 10대 '] = move_df.iloc[:,19]+move_df.iloc[:,20]
    move_df_clean['여성 20대 '] = move_df.iloc[:,21]+move_df.iloc[:,22]
    move_df_clean['여성 30대 '] = move_df.iloc[:,23]+move_df.iloc[:,24]
    move_df_clean['여성 40대 '] = move_df.iloc[:,25]+move_df.iloc[:,26]
    move_df_clean['여성 50대 '] = move_df.iloc[:,27]+move_df.iloc[:,28]
    move_df_clean['여성 60대 '] = move_df.iloc[:,29]+move_df.iloc[:,30]
    move_df_clean['여성 70대 이상'] = move_df.iloc[:,31]

    move_df_age_mf = move_df_clean.copy()


    #행열 전치
    move_df_age_mf_t = pd.melt(move_df_age_mf, id_vars =['기준일ID', '행정동코드', '시간대구분'], var_name = '연령코드', value_name = '인구수')


    #연령코드에 총생활인구수 삭제
    move_df_age_mf_t = move_df_age_mf_t.loc[move_df_age_mf_t['연령코드'] != '총생활인구수'].reset_index().drop('index',axis = 1)


    #성별, 연령대 분리
    move_df_age_mf_t['성별'] = move_df_age_mf_t['연령코드'].apply(lambda x: x.split(' ')[0])
    move_df_age_mf_t['연령대'] = move_df_age_mf_t['연령코드'].replace(['남성 ','여성 '],'',regex = True)

    #행정동법정동 매핑
    map_move_df_age_mf_t = pd.merge(move_df_age_mf_t,mapping_bh_dong)


    #행정동/법정동 코드 되돌리기
    map_move_df_age_mf_t['행정동코드'] = map_move_df_age_mf_t['행정동코드']*100
    map_move_df_age_mf_t['법정동코드'] = map_move_df_age_mf_t['법정동코드']*100


    # 필요 컬럼만 분리
    map_move_df_age_mf_t = map_move_df_age_mf_t[['기준일ID','시군구명','법정동코드','법정동명','법정동개수','시간대구분','성별','연령대','인구수']]

    # 공통인덱스로 sum 후 인덱스 reset
    clean_move_df_wan = map_move_df_age_mf_t.groupby(['기준일ID','시군구명','법정동코드','법정동명','시간대구분','성별','연령대','법정동개수']).sum().reset_index()


    #법정동 개수만큼 인구수 나누기
    clean_move_df_wan['생활인구수'] = clean_move_df_wan['인구수']/clean_move_df_wan['법정동개수']

    #일평균내기
    clean_move_df_wan = clean_move_df_wan.groupby(['시군구명','법정동코드','법정동명','시간대구분','성별','연령대']).mean().reset_index()
    clean_move_df_wan = clean_move_df_wan[['시군구명','법정동코드','법정동명','시간대구분','성별','연령대','생활인구수']]
    
    #시간대평균내기
    clean_move_df_wan = clean_move_df_wan.groupby(['시군구명','법정동코드','법정동명','성별','연령대']).mean().reset_index()
    clean_move_df_wan = clean_move_df_wan[['시군구명','법정동코드','법정동명','성별','연령대','생활인구수']]
    
    #저장
    clean_move_df_wan.to_csv('cleanmovedf'+i+'.csv')